# EPA1361 - Model-Based Decision Making

## Multi-model analysis

This exercise uses a simple version of the [Lotka-Volterra predator-prey equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations) to show how the EMA Workbench can be used for a
multi-model analysis, in addition to typical parametric/structural uncertainties. This will let you test the connectors provided in the Workbench for Excel, NetLogo, and Vensim / PySD; we'll also use the models for the sensitivity analysis exercise in week 3.

**Assignment**
Using the three model files provided and the Python function below, define model objects for each implementation (Excel, NetLogo, Vensim/PySD, and Python), and test them using a single ensemble. Use 50 experiments sampled from the parameters below (so that each experiment will be executed for the 4 models, for a total of 200), and retrieve outputs for the _TIME_, _predators_, and _prey_ variables.
   * Excel and Vensim are only supported on Windows
   * Vensim requires the DSS version of Vensim
   * Netlogo supoprt depends on [jpype](http://jpype.readthedocs.io/en/latest/install.html) and [pynetlogo](https://pynetlogo.readthedocs.io/en/latest/). Also, if you don't have NetLogo installed, please get [NetLogo 6.3.0](https://ccl.northwestern.edu/netlogo/download.shtml)
   * for pysd, see [its documentation](http://pysd.readthedocs.io/en/master/installation.html)
   * If possible try to work with all model versions, but even 2 or 3 (pure python and something else should be sufficient).


|Parameter	|Range or value	        |
|-----------|--------------:|
|prey_birth_rate    	|0.015 – 0.035	|
|predation_rate|0.0005 – 0.003 	|
|predator_efficiency     	|0.001 – 0.004	    |
|predator_loss_rate	    |0.04 – 0.08	    |
|Final time	    |365	    |
|dt	    |0.25	    |

* Note that your EMA Workbench installation includes [example scripts](https://github.com/quaquel/EMAworkbench/tree/master/ema_workbench/examples) for the different connectors. The different model objects follow a similar syntax but will need to be slightly adjusted depending on the software (e.g. to specify the NetLogo run length or the sheet name in Excel).
  * This [tutorial](https://emaworkbench.readthedocs.io/en/latest/basic_tutorial.html) also shows a simple model in Python, Vensim and Excel connected to the workbench.

* These model objects can be used with a replication functionality (for instance to test the effect of stochastic uncertainty in a NetLogo model), which repeats a given experiment over multiple replications. You can use a single replication in this exercise as the models are not stochastic. By default, each outcome array will then have a shape of (# experiments, # replications, # time steps). Try adapting the outcome arrays so that they can be used with the _lines_ plotting function of the Workbench, and plot the results grouped by model.

* To check the graphical results, find the maximum absolute error of the time series you obtained for the _prey_ variable in the Excel, NetLogo, and Vensim/PySD models, relative to the Python function.

In [1]:
# Some imports you may need
import numpy as np
import matplotlib.pyplot as plt

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, perform_experiments, ema_logging)

from ema_workbench.connectors.netlogo import NetLogoModel
from ema_workbench.connectors.excel import ExcelModel
from ema_workbench.connectors.pysd_connector import PysdModel

from ema_workbench.em_framework.samplers import LHSSampler
from ema_workbench.em_framework.salib_samplers import MorrisSampler, SobolSampler

from ema_workbench.analysis.plotting import lines, Density

/Users/inez/Desktop/tu/epa/jaar 1/q4/epa141A_open-master/.venv/lib/python3.11/site-packages/ema_workbench/connectors/__init__.py:19: ImportWarning: vensim connector not available
  warnings.warn("vensim connector not available", ImportWarning)
/Users/inez/Desktop/tu/epa/jaar 1/q4/epa141A_open-master/.venv/lib/python3.11/site-packages/ema_workbench/connectors/__init__.py:34: ImportWarning: simio connector not available
  warnings.warn("simio connector not available", ImportWarning)


In [2]:
# Import the Python function
from model.pred_prey import PredPrey

In [3]:
# Define uncertainties and outcomes


# Define model objects for the different implementations


In [6]:
from ema_workbench import Model, RealParameter, TimeSeriesOutcome
from ema_workbench.connectors.netlogo import NetLogoModel

# Define the Python model
python_model = Model("PredPreyPython", function=PredPrey)

# Define the NetLogo model
# netlogo_model = NetLogoModel("PredPreyNetLogo", wd='model/PredPrey.nlogo')
netlogo_model = NetLogoModel("PredPreyNetLogo", wd='model', model_file='PredPrey.nlogo')

# Define uncertainties (parameters to be varied)
uncertainties = [
    RealParameter('prey_birth_rate', 0.015, 0.035),
    RealParameter('predation_rate', 0.0005, 0.003),
    RealParameter('predator_efficiency', 0.001, 0.004),
    RealParameter('predator_loss_rate', 0.04, 0.08),
    RealParameter('dt', 0.25, 0.25),
]

# Assign uncertainties to both models
python_model.uncertainties = uncertainties
netlogo_model.uncertainties = uncertainties

# Define the outcomes (what you want to measure)
outcomes = [
    TimeSeriesOutcome('TIME'),
    TimeSeriesOutcome('predators'),
    TimeSeriesOutcome('prey')
]

# Assign outcomes to both models
python_model.outcomes = outcomes
netlogo_model.outcomes = outcomes

# Now, create the model objects for both Python and NetLogo
models = [python_model, netlogo_model]

# Perform experiments
experiments, outcomes = perform_experiments(models, 50)

# Plotting or analysis after experiments
# Plotting the results (grouped by model)
for outcome in outcomes.keys():
    lines(experiments, outcomes, outcomes_to_show=[outcome], density='hist')

plt.show()



  0%|                                                  | 0/100 [00:00<?, ?it/s]

EMAError: Some fatal error has occurred while running the experiments, not all runs have completed. Expected 100, got 0

In [7]:
# Test with only the Python model
python_model = Model("PredPreyPython", function=PredPrey)
python_model.uncertainties = uncertainties
python_model.outcomes = outcomes

# Run a simple experiment with 10 scenarios
experiments, outcomes = perform_experiments(python_model, 10)

# Check the outcome
for outcome in outcomes.keys():
    lines(experiments, outcomes, outcomes_to_show=[outcome], density='hist')

plt.show()




  0%|                                                   | 0/10 [00:00<?, ?it/s]

EMAError: Some fatal error has occurred while running the experiments, not all runs have completed. Expected 10, got 0

In [15]:
params = {
    'prey_birth_rate': 0.02,
    'predation_rate': 0.001,
    'predator_efficiency': 0.002,
    'predator_loss_rate': 0.05,
    'dt': 0.25
}

output = PredPrey(params)
print(output)

TypeError: unsupported operand type(s) for *: 'dict' and 'float'